In [1]:
#import libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import sqlite3
from datetime import datetime

In [2]:
url = 'https://web.archive.org/web/20230908091635 /https://en.wikipedia.org/wiki/List_of_largest_banks'
csv_path = r'C:\Users\esraa\Desktop\PythonProjects\ETL_WorldLargestBanks_Jupiter/Largest_banks_data.csv'
db_name = r'C:\Users\esraa\Desktop\PythonProjects\ETL_WorldLargestBanks_Jupiter/Banks.db'
#db_name = 'Banks.db'
table_name = 'Largest_banks'

In [3]:
''' This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing'''

def log_progress(message):
    
    #timestamp_format --> determines the formatting of the time &date
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    #now --> capture the current time by calling datetime 
    now = datetime.now() # get current timestamp 
    timestamp = now.strftime(timestamp_format)
    #pull that information together (by opening a file &writing the information to the file)
    # 'a' ---> all the data written will be appended to the existing information
    with open("C:/Users/esraa/Desktop/PythonProjects/ETL_WorldLargestBanks_Jupiter/code_log.txt","a") as f:
        #we are then able to attach a timestamp to each part of the process of when it begins &when it has completed 
        f.write(timestamp + ' : ' + message + '\n')

In [4]:
#        Task           -->       Log message on completion

#Declaring known values --> Preliminaries complete. Initiating ETL process
# Log the initialization of the ETL process 
log_progress('Preliminaries complete. Initiating ETL process')

In [5]:
#################################################### Extraction ####################################################
table_attribs = ['Name', 'MC_USD_Billion']


#Extract the web page as text and store in a variable
html_page = requests.get(url).text
#create a BeautifulSoup object using the BeautifulSoup constructor
##Parse the text into an HTML object
data = BeautifulSoup(html_page, 'html.parser')


#get the body of all the tables in the web page
tables = data.find_all('tbody')
#get all the rows of the first table
rows = tables[0].find_all('tr')



''' This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. '''

def extract(url, table_attribs):
    #Create an empty pandas DataFrame named df with columns as the table_attribs
    df = pd.DataFrame(columns=table_attribs)

    for row in rows:
        #Check the contents of each row having attribute ‘td’, (should not be empty)#
        col = row.find_all('td')
        if len(col)!=0:
        #Store all entries matching the conditions to a dictionary with keys the same as table_attribs#
            #data_dict = {"Name": col[1].text, "MC_USD_Billion": col[2].contents[0]}
            data_dict = {"Name": col[1].find_all('a')[1]['title'], "MC_USD_Billion": col[2].contents}
            df1 = pd.DataFrame(data_dict, index=[0])
            #Append all these dictionaries one by one to the dataframe#
            #remove the last character from the Market Cap column contents'\n' #
            df = pd.concat([df,df1], ignore_index=True).replace('\n', '', regex=True)
    return df 

In [6]:
#Call extract() function --> Data extraction complete. Initiating Transformation process

#call the extract_data function
##(The data received from this step will then be transferred to the 2nd step of transforming, 
##after this has completed the data is then loaded into the target file)


#before & after every step the time and date for start and completion has been added

In [7]:
df = extract(url, table_attribs)
df

,Name,MC_USD_Billion
0,JPMorgan Chase,432.92
1,Bank of America,231.52
2,Industrial and Commercial Bank of China,194.56
3,Agricultural Bank of China,160.68
4,HDFC Bank,157.91
5,Wells Fargo,155.87
6,HSBC,148.90
7,Morgan Stanley,140.83
8,China Construction Bank,139.82
9,Bank of China,136.81


In [8]:
log_progress('Data extraction complete. Initiating Transformation process')

In [9]:
################################################################################################################

In [10]:
################################################ Transformation ################################################

In [11]:
exchange_rate = pd.read_csv(r"C:\Users\esraa\Desktop\PythonProjects\ETL_WorldLargestBanks_Jupiter\exchange_rate.csv")
exchange_rate

,Currency,Rate
0,EUR,0.93
1,GBP,0.80
2,INR,82.95


In [12]:
#convert the contents of the exchange rate CSV file to a dictionary so that 
##the contents of the first columns are the keys to the dictionary and 
##the contents of the second column are the corresponding values

exchange_rate = exchange_rate.set_index('Currency').to_dict()['Rate']
exchange_rate

{'EUR': 0.93, 'GBP': 0.8, 'INR': 82.95}

In [13]:
df["MC_USD_Billion"]

0    432.92
1    231.52
2    194.56
3    160.68
4    157.91
5    155.87
6    148.90
7    140.83
8    139.82
9    136.81
Name: MC_USD_Billion, dtype: object

In [14]:
#Convert the contents of the 'MC_USD_Billion' column of df dataframe from currency format to floating numbers

#convert the dataframe column 'MC_USD_Billion' to a list 
#to iterate over the contents of the list and use split() and join() functions to convert the currency text into numerical text

In [15]:
#convert the dataframe column'MC_USD_Billion' to a list
MC_USD_Billion = df["MC_USD_Billion"].tolist()
MC_USD_Billion

['432.92',
 '231.52',
 '194.56',
 '160.68',
 '157.91',
 '155.87',
 '148.90',
 '140.83',
 '139.82',
 '136.81']

In [16]:
#Typecast the currency format to float
MC_USD_Billion = [float("".join(x.split('.'))) for x in MC_USD_Billion]
MC_USD_Billion

[43292.0,
 23152.0,
 19456.0,
 16068.0,
 15791.0,
 15587.0,
 14890.0,
 14083.0,
 13982.0,
 13681.0]

In [17]:
''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''

def transform(df, exchange_rate):
    #transform 'MC_USD_Billion' from USD to GBP, EUR, INR as per the exchange_rate info in the CSV file#
    #and round it to 2 decimal places by using the numpy.round() function#
    df['MC_USD_Billion'] = [np.round(x ,2) for x in MC_USD_Billion]
    df['MC_GBP_Billion'] = [np.round(x * exchange_rate['GBP'],2) for x in MC_USD_Billion]
    df['MC_EUR_Billion'] = [np.round(x * exchange_rate['EUR'],2) for x in MC_USD_Billion]
    df['MC_INR_Billion'] = [np.round(x * exchange_rate['INR'],2) for x in MC_USD_Billion]
    return df

In [18]:
df = transform(df, exchange_rate)
df

,Name,MC_USD_Billion,MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
0,JPMorgan Chase,43292.0,34633.6,40261.56,3591071.40
1,Bank of America,23152.0,18521.6,21531.36,1920458.40
2,Industrial and Commercial Bank of China,19456.0,15564.8,18094.08,1613875.20
3,Agricultural Bank of China,16068.0,12854.4,14943.24,1332840.60
4,HDFC Bank,15791.0,12632.8,14685.63,1309863.45
5,Wells Fargo,15587.0,12469.6,14495.91,1292941.65
6,HSBC,14890.0,11912.0,13847.70,1235125.50
7,Morgan Stanley,14083.0,11266.4,13097.19,1168184.85
8,China Construction Bank,13982.0,11185.6,13003.26,1159806.90
9,Bank of China,13681.0,10944.8,12723.33,1134838.95


In [19]:
log_progress('Data transformation complete. Initiating Loading process')

In [20]:
############################################################################################################################

In [21]:
#################################################### Loading ####################################################
#save the transformed dataframe to a CSV file
##pass the dataframe df and the CSV file path to the function load_to_csv() 

''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''

def load_to_csv(df, csv_path):
    df.to_csv(csv_path)
#Calling the function    
load_to_csv(df, csv_path)


log_progress('Data saved to CSV file')

In [22]:
#Load the transformed dataframe to an SQL database server as a table
##This needs to be implemented in the function load_to_db(), which accepts the dataframe df, 
###the connection object to the SQL database conn, and the table name variable table_name to be used


#Initiate SQLite3 connection --> SQL Connection initiated.
sql_connection = sqlite3.connect(db_name)
log_progress('SQL Connection initiated.')



''' This function saves the final data frame to a database
    table with the provided name. Function returns nothing.'''
def load_to_db(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)
#Calling the function 
load_to_db(df, sql_connection, table_name)


#Call load_to_db() --> Data loaded to Database as a table, Executing queries
log_progress('Data loaded to Database as a table, Executing queries')

In [23]:
#Run queries on Database
#run_query():accepts the query statement, and the SQLite3 Connection object, and generates the output of the query

''' This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. '''
def run_query(query_statement, sql_connection):
    query_output = pd.read_sql(query_statement, sql_connection)

In [24]:
#Print the contents of the entire table
query_statement = f"SELECT * FROM {table_name}"
query_output = pd.read_sql(query_statement, sql_connection)
query_output

,Name,MC_USD_Billion,MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
0,JPMorgan Chase,43292.0,34633.6,40261.56,3591071.40
1,Bank of America,23152.0,18521.6,21531.36,1920458.40
2,Industrial and Commercial Bank of China,19456.0,15564.8,18094.08,1613875.20
3,Agricultural Bank of China,16068.0,12854.4,14943.24,1332840.60
4,HDFC Bank,15791.0,12632.8,14685.63,1309863.45
5,Wells Fargo,15587.0,12469.6,14495.91,1292941.65
6,HSBC,14890.0,11912.0,13847.70,1235125.50
7,Morgan Stanley,14083.0,11266.4,13097.19,1168184.85
8,China Construction Bank,13982.0,11185.6,13003.26,1159806.90
9,Bank of China,13681.0,10944.8,12723.33,1134838.95


In [25]:
#Print the average market capitalization of all the banks in Billion USD.
query_statement = f"SELECT AVG(MC_GBP_Billion) FROM {table_name}"
query_output = pd.read_sql(query_statement, sql_connection)
query_output

,AVG(MC_GBP_Billion)
0,15198.56


In [26]:
#Print only the names of the top 5 banks
query_statement = f"SELECT Name from {table_name} LIMIT 5"
query_output = pd.read_sql(query_statement, sql_connection)
query_output

,Name
0,JPMorgan Chase
1,Bank of America
2,Industrial and Commercial Bank of China
3,Agricultural Bank of China
4,HDFC Bank


In [27]:
#Calling the function 
run_query(query_statement, sql_connection)   

#Call run_query()   --> Process Complete.
log_progress('Process Complete.')

In [28]:
#Close SQLite3 connection --> Server Connection closed
sql_connection.close()
print('Server Connection closed')

Server Connection closed


In [29]:
##########################################################################################################

In [30]:
########################################### Verify log entries ###########################################
#        Task           -->       Log message on completion

#Declaring known values --> Preliminaries complete. Initiating ETL process.
# Log the initialization of the ETL process 
log_progress('Preliminaries complete. Initiating ETL process')


#Call extract() function --> Data extraction complete. Initiating Transformation process.
#call the extract_data function
#(The data received from this step will then be transferred to the 2nd step of transforming, after this has completed the data
# is then loaded into the target file)
#before & after every step the time and date for start and completion has been added
df = extract(url, table_attribs)
log_progress('Data extraction complete. Initiating Transformation process')


#Call transform() function --> Data transformation complete. Initiating loading process.
df = transform(df, exchange_rate)
log_progress('Data transformation complete. Initiating loading process')

#Call load_to_csv()  --> Data saved to CSV file.
load_to_csv(df, csv_path)
log_progress('Data saved to CSV file')

#Initiate SQLite3 connection --> SQL Connection initiated.
sql_connection = sqlite3.connect(db_name)
log_progress('SQL Connection initiated.')

#Call load_to_db() --> Data loaded to Database as table. Running the query.
load_to_db(df, sql_connection, table_name)
log_progress('Data loaded to Database as a table, Executing queries')

#Call run_query() *  --> Process Complete.
#Query: Display only the entries with more than a 100 billion USD economy
run_query(query_statement, sql_connection)

log_progress('Process Complete.')

#Close SQLite3 connection --> -
sql_connection.close()
print('Server Connection closed')

Server Connection closed


In [31]:
###########################################################################################################################